In [1]:
import numpy as np
import gc
import timeit
import torch 

# Concat large arrays 

In [2]:
real_shape = (206928225, 100)
testshape = (int(206928225/100), 100)
testshape
test = True
no_tries = 3

In [3]:
if test:
    shape = testshape
else:
    shape = real_shape

In [4]:
def fast_concat(se,pe,oe):
    
    assert se.shape == pe.shape, "Error! fast_concat with differing shapes"
    assert se.shape == oe.shape, "Error! fast_concat with differing shapes"
       
    x = np.empty((se.shape[0],se.shape[1]*3),dtype=np.float32)
    x[:,0:100] =se
    x[:,100:200] = pe
    x[:,200:] = oe
    
    return x

def insert_row_into_array_np(arr,row,col_ix):
    col = np.repeat(row,len(arr)).reshape((len(arr),col_ix[1]-col_ix[0]))
    arr[:,col_ix[0]:col_ix[1]] = col
    return arr

def insert_row_into_array_zero_np(arr,row,col_ix):
    col = np.repeat(row,len(arr)).reshape((len(arr),col_ix[1]-col_ix[0]))
    arr[:,col_ix[0]:col_ix[1]] = 0
    arr[:,col_ix[0]:col_ix[1]] += col
    
    return arr

def insert_row_into_array_loop_np(arr,row,col_ix):
    for i in range(len(arr)):
        arr[:,col_ix[0]:col_ix[1]][i] = row
    return arr

def insert_row_into_array_loop_zero_np(arr,row,col_ix):
    arr[:,col_ix[0]:col_ix[1]] = 0
    for i in range(len(arr)):
        arr[:,col_ix[0]:col_ix[1]][i] += row
    return arr

In [5]:
def insert_row_into_array_torch(arr,row,col_ix):
    col = row.repeat(len(arr),1)
    arr[:,col_ix[0]:col_ix[1]] = col
    return arr

def insert_row_into_array_zero_torch(arr,row,col_ix):
    col = row.repeat(len(arr),1)
    arr[:,col_ix[0]:col_ix[1]] = 0
    arr[:,col_ix[0]:col_ix[1]] += col
    return arr

def insert_row_into_array_loop_torch(arr,row,col_ix):
    for i in range(len(arr)):
        arr[:,col_ix[0]:col_ix[1]][i] = row
    return arr

def insert_row_into_array_loop_zero_torch(arr,row,col_ix):
    arr[:,col_ix[0]:col_ix[1]] = 0
    for i in range(len(arr)):
        arr[:,col_ix[0]:col_ix[1]][i] += row
    return arr

In [6]:
testrow = np.zeros(shape[1])
testarray = np.empty((shape[0],shape[1]*3))

try_np = f"insert_row_into_array_np(testarray,testrow,(100,200));gc.collect()"
try_np_zero= f"insert_row_into_array_zero_np(testarray,testrow,(100,200));gc.collect()"
try_np_loop= f"insert_row_into_array_loop_np(testarray,testrow,(100,200));gc.collect()"
try_np_loop_zero = f"insert_row_into_array_loop_zero_np(testarray,testrow,(100,200));gc.collect()"

## Time numpy functions

In [7]:
timeit.timeit(try_np, number=no_tries,globals=globals())

KeyboardInterrupt: 

In [ ]:
timeit.timeit(try_np_zero, number=no_tries,globals=globals())

In [ ]:
timeit.timeit(try_np_loop, number=no_tries,globals=globals())

In [ ]:
timeit.timeit(try_np_loop_zero, number=no_tries,globals=globals())

## Try torch functions

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
testrow = torch.tensor(testrow).to(device)
testarray = torch.tensor(testarray).to(device)

try_torch = f"insert_row_into_array_torch(testarray,testrow,(100,200));gc.collect()"
try_torch_zero= f"insert_row_into_array_zero_torch(testarray,testrow,(100,200));gc.collect()"
try_torch_loop= f"insert_row_into_array_loop_torch(testarray,testrow,(100,200));gc.collect()"
try_torch_loop_zero= f"insert_row_into_array_loop_zero_torch(testarray,testrow,(100,200));gc.collect()"

In [ ]:
timeit.timeit(try_torch, number=no_tries,globals=globals())

In [ ]:
timeit.timeit(try_torch_zero, number=no_tries,globals=globals())

In [ ]:
timeit.timeit(try_torch_loop, number=no_tries,globals=globals())

In [ ]:
timeit.timeit(try_torch_loop_zero, number=no_tries,globals=globals())

In [ ]:
# pytorch model
import torchmetrics
import torch
from torch import nn 


class ClassifierSimple(torch.nn.Module):
    def __init__(self,input_dim=300,hidden_size=64):
        super(ClassifierSimple, self).__init__()
        
        
        self.layers = nn.Sequential(
                # flatten input if necessary
                nn.Flatten(),
                nn.Linear(input_dim,hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size,1)
        )
        
        self.output_activation = nn.Sigmoid()
                
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
        
    
    def forward(self,x):        
        
        return self.layers(x)
    def predict(self,x):
        x.to(self.device)
        
        return self.output_activation(self.layers(x))
    def predict_numpy(self,x):
        x = torch.tensor(x)
        x.to(self.device)
        return self.output_activation(self.layers(x)).detach().cpu().numpy()
        
    


# Test Prediction function

In [ ]:
model = ClassifierSimple()
model = model.to(device)

In [ ]:
test = torch.empty((shape[0],shape[1]*3)).to(device)

In [ ]:
timeit.timeit(f"model(test);gc.collect()", number=10,globals=globals())

In [ ]:
timeit.timeit(f"model.predict_numpy(test);gc.collect()", number=10,globals=globals())

No Difference on cpu ... need to check on gpu

In [1]:
import torch

In [2]:
x = torch.zeros(10000,300)

In [3]:
row = torch.ones(100).reshape(1,100)

In [9]:
torch.arange(10)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
x[:,100:200].index_fill(0,torch.arange(len(x)),row)

RuntimeError: index_fill_ only supports a 0-dimensional value tensor, but got tensor with 2 dimension(s).

# Test to(device)

In [104]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [103]:
x = torch.ones(10000,500)
